## 시설물 유지보수대장 목록

In [2]:
from common import commonFunc as cf
import pandas as pd
import pickle

pd.set_option('display.max_columns', None)
metadata = pd.read_excel("../input/datalake_meta22.xlsx")

SITENAME = "건설사업정보시스템"
DATANAME= "시설물 유지보수대장 목록"
with open("../input/calsapikey.pickle","rb") as fr:
    APIKEY = pickle.load(fr)

targetData = metadata.loc[metadata.자료명==DATANAME]
preSetFolder = targetData["저장폴더"].values[0]

preSetFolder

'../output/건설사업정보시스템/시설물 유지보수대장 목록/selectIoFmCwkRegsList.csv'

In [3]:
# 개별 시설물 유지보수대장목록 수
g_totalCount = 0

In [4]:
##### 파라미터 설정 #####
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
SERVICENAME

'selectIoFmCwkRegsList'

In [6]:
REQPARAM = targetData["요청변수"].values[0]
REQPARAM = REQPARAM.split(",")
PRIMARYKEY = targetData["기본키"].values[0]
REQPARAM

['serviceKey',
 'type',
 'pageNo',
 'numOfRows',
 'sortField',
 'sortOrder',
 'searchCwkEdYr',
 'searchCwkNm']

In [7]:
# 페이지번호 설정
PAGEYN=1
if REQPARAM.count("pageNo") == 0:
    PAGEYN = 0
else:
    PAGEYN = 1
### 기본정보 설정 및 파라미터 설정 ###
PAGEYN

1

In [8]:
JSONKEY = "items"
DUMMY = 0

In [9]:
newParam = []
mode = 2
if newParam==[]:
    mode=2
    print("{} 정보 new 모드 {} ".format(DATANAME, mode))

시설물 유지보수대장 목록 정보 new 모드 2 


## 시설물 유지보수대장 목록 전용 스크랩 함수

In [10]:

numOfRows = 1000
pageNo = 1

In [11]:
class noDataFound(Exception): pass  # declare a label


In [14]:
resultDfMerged = pd.DataFrame()
resultDf = pd.DataFrame()
pageList = [] # 4/28 추가
APICALL = 0 #4/28 추가

i = 0
try:
    for i in range(0, 1):
#         pageNo = 1
#         global g_totalCount
#         g_totalCount = 0
#         print("i=",i)

        # 25번 도로시설물 관리현황 정보의 검색결과 패턴을 비교하였을때 items에만 결과를 줄 것으로 예상
        BASEPARAM={"serviceKey":APIKEY[0],"pageNo": pageNo, 'numOfRows':numOfRows, "type":"json"}
#             resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY)
        scrapyResult = cf.scrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN,APIKEY,APICALL) #신규 4/28
        resultDf = scrapyResult[0]
        pageList.append( scrapyResult[1] )
        APICALL = scrapyResult[2] 
        if resultDf.empty: # 정상 데이터가 없는 경우
            print("NO DATA FOUND, CHECK CALS API")
            raise noDataFound()
        print(resultDf)
        resultDfMerged = resultDfMerged.append(resultDf)
            
            
#             else:
#                 resultDfMerged = resultDfMerged.append(resultDf) 
#                 g_totalCount -= numOfRows
#                 print("g_totalCount : ", g_totalCount, " numOfRows : ", numOfRows)
#                 if(g_totalCount <= 0):
#                     break
#                 pageNo += 1
        
except noDataFound:
    print("exception handled")
    
        

1 page scraping start apicall iter: 1 / used 49099D1D-E565-48E1-8A6F-D5F47135EA34
1 page is empty
dataframe시설물 유지보수대장 목록, param:{'serviceKey': '49099D1D-E565-48E1-8A6F-D5F47135EA34', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'} rows: 0 completed
NO DATA FOUND, CHECK CALS API
exception handled


In [18]:
resultDfMerged

""


In [16]:
cf.savedata(resultDfMerged, SITENAME,DATANAME,SERVICENAME,mode)

시설물 유지보수대장 목록 save compled


In [17]:
resultDfMerged.shape

(0, 0)

In [19]:
# 파라미터 저장 : 요청 키가 없는 API이므로, pageList만 저장

# 기존사용된 파라미터 정보에 추가 파라미터 append 후 저장
# ptlcmnoList = orgParam + ptlcmnoList
# 파라미터 없는경우 ptlcmnoList = []
# ptlcmnoList = []
paramList = [pageList]
# # 최종 파라미터 저장
cf.saveparam(paramList, SITENAME,DATANAME,SERVICENAME)